<a href="https://colab.research.google.com/github/valvarezma/vision_artificial/blob/main/template_maching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


Install and load the nvcc4jupyter extension, which will allow us to run CUDA C++ code using cell magics. See [the documentation](https://nvcc4jupyter.readthedocs.io/en/latest/usage.html) for details.

In [ ]:
%%capture
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpc909e385".


## Code

### Imports

In [ ]:
import os
import subprocess
from IPython.display import Image

### Compiler arguments
These are options that you need to pass to the compiler in order to link with the OpenCV library.

In [ ]:
COMPILER_ARGS = subprocess.check_output(["pkg-config", "--cflags", "--libs", "opencv4"]).decode().strip()
print(COMPILER_ARGS)

-I/usr/include/opencv4 -lopencv_stitching -lopencv_alphamat -lopencv_aruco -lopencv_barcode -lopencv_bgsegm -lopencv_bioinspired -lopencv_ccalib -lopencv_dnn_objdetect -lopencv_dnn_superres -lopencv_dpm -lopencv_face -lopencv_freetype -lopencv_fuzzy -lopencv_hdf -lopencv_hfs -lopencv_img_hash -lopencv_intensity_transform -lopencv_line_descriptor -lopencv_mcc -lopencv_quality -lopencv_rapid -lopencv_reg -lopencv_rgbd -lopencv_saliency -lopencv_shape -lopencv_stereo -lopencv_structured_light -lopencv_phase_unwrapping -lopencv_superres -lopencv_optflow -lopencv_surface_matching -lopencv_tracking -lopencv_highgui -lopencv_datasets -lopencv_text -lopencv_plot -lopencv_ml -lopencv_videostab -lopencv_videoio -lopencv_viz -lopencv_wechat_qrcode -lopencv_ximgproc -lopencv_video -lopencv_xobjdetect -lopencv_objdetect -lopencv_calib3d -lopencv_imgcodecs -lopencv_features2d -lopencv_dnn -lopencv_flann -lopencv_xphoto -lopencv_photo -lopencv_imgproc -lopencv_core


### Template Matching

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%cuda --compiler-args "$COMPILER_ARGS"
#include <iostream>
#include <opencv2/opencv.hpp>

__global__ void templateMatching(const uchar* input, const uchar* template_img, float* result, int input_rows, int input_cols, int template_rows, int template_cols) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < input_rows - template_rows && col < input_cols - template_cols) {
        float sum = 0.0f;
        float input_mean = 0.0f;
        float template_mean = 0.0f;

        // Calcular la media de la imagen de entrada y de la plantilla
        for (int i = 0; i < template_rows; ++i) {
            for (int j = 0; j < template_cols; ++j) {
                input_mean += input[(row + i) * input_cols + (col + j)];
                template_mean += template_img[i * template_cols + j];
            }
        }
        input_mean /= (template_rows * template_cols);
        template_mean /= (template_rows * template_cols);

        // Calcular la correlación cruzada normalizada (NCC)
        for (int i = 0; i < template_rows; ++i) {
            for (int j = 0; j < template_cols; ++j) {
                float input_pixel = input[(row + i) * input_cols + (col + j)];
                float template_pixel = template_img[i * template_cols + j];
                sum += (input_pixel - input_mean) * (template_pixel - template_mean);
            }
        }
        result[row * (input_cols - template_cols) + col] = sum;
    }
}

int main() {
    cv::Mat input_image = cv::imread("/content/drive/MyDrive/template_matching/imagen_a.jpg", cv::IMREAD_GRAYSCALE);
    cv::Mat template_image = cv::imread("/content/drive/MyDrive/template_matching/imagen_b.jpg", cv::IMREAD_GRAYSCALE);

    if (input_image.empty() || template_image.empty()) {
        std::cerr << "Error: No se pudieron cargar las imágenes." << std::endl;
        return 1;
    }

    int input_rows = input_image.rows;
    int input_cols = input_image.cols;
    int template_rows = template_image.rows;
    int template_cols = template_image.cols;

    uchar *d_input, *d_template;
    float *d_result;

    cudaMalloc(&d_input, input_rows * input_cols * sizeof(uchar));
    cudaMalloc(&d_template, template_rows * template_cols * sizeof(uchar));
    cudaMalloc(&d_result, (input_rows - template_rows) * (input_cols - template_cols) * sizeof(float));

    cudaMemcpy(d_input, input_image.data, input_rows * input_cols * sizeof(uchar), cudaMemcpyHostToDevice);
    cudaMemcpy(d_template, template_image.data, template_rows * template_cols * sizeof(uchar), cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((input_cols - template_cols + threadsPerBlock.x - 1) / threadsPerBlock.x, (input_rows - template_rows + threadsPerBlock.y - 1) / threadsPerBlock.y);

    templateMatching<<<numBlocks, threadsPerBlock>>>(d_input, d_template, d_result, input_rows, input_cols, template_rows, template_cols);
    cudaDeviceSynchronize();

    float* result = new float[(input_rows - template_rows) * (input_cols - template_cols)];
    cudaMemcpy(result, d_result, (input_rows - template_rows) * (input_cols - template_cols) * sizeof(float), cudaMemcpyDeviceToHost);

    // Encontrar la ubicación de la mejor coincidencia
    float max_val = -FLT_MAX;
    cv::Point best_match;
    for (int i = 0; i < input_rows - template_rows; ++i) {
        for (int j = 0; j < input_cols - template_cols; ++j) {
            if (result[i * (input_cols - template_cols) + j] > max_val) {
                max_val = result[i * (input_cols - template_cols) + j];
                best_match = cv::Point(j, i);
            }
        }
    }

    // Dibujar el recuadro en la imagen de entrada
    cv::rectangle(input_image, best_match, cv::Point(best_match.x + template_cols, best_match.y + template_rows), cv::Scalar(0, 255, 0), 2);

    // Mostrar la imagen con el recuadro

    cv::imwrite("/content/drive/MyDrive/template_matching/image_cuadro.jpg", input_image);
    cv::waitKey(0);
    cv::destroyAllWindows();

    delete[] result;
    cudaFree(d_input);
    cudaFree(d_template);
    cudaFree(d_result);

    return 0;
}
